In [14]:
# 2 . Morphometry on buildings using french BD TOPO
# http://emc2-dut.org/
# BD TOPO® décembre 2023 Tous Thèmes par département format GeoPackage projection légale
# Webpage : https://geoservices.ign.fr/bdtopo#telechargementgpkgreg
# Author : Perez Joan*  (2024) 
# * UMR 7300 ESPACE-CNRS, Université Côte d'Azur, Nice, France.
#   Contact: Joan.PEREZ@univ-cotedazur.fr 

In [11]:
import pandas as pd
import geopandas as gpd
import numpy as np
import momepy
import libpysal
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Load data
# Specify the location of subset
gpkg = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\ICCSA_2024 data and codes\\DPC_59.gpkg"
building = gpd.read_file(gpkg, layer="building")

In [12]:
# 1  Fill missing values of number of floors/height based on attribute values (non mandatory)
# Fill Missing Height
building['hauteur'] = building['hauteur'].fillna(building['altitude_minimale_toit'] - building['altitude_minimale_sol'])
# checks if the 'hauteur' column is NaN and if the 'nombre_d_etages' is not NaN.
# If both conditions are met : multiplies the value of 'nombre_d_etages' by 3 and assigns it to 'hauteur'
building['hauteur'] = building.apply(lambda row: row['nombre_d_etages'] * 3 if pd.isna(row['hauteur'])
                                     and not pd.isna(row['nombre_d_etages']) else row['hauteur'], axis=1)
# checks if the 'hauteur' column is NaN and if the 'nombre_d_etages' is not NaN.
# If both conditions are met : multiplies the value of 'nombre_d_etages' by 3 and assigns it to 'hauteur'
building['nombre_d_etages'] = building.apply(lambda row: round(row['hauteur'] / 3) if pd.isna(row['nombre_d_etages'])
                                             and not pd.isna(row['hauteur']) else row['nombre_d_etages'], axis=1)
print('Number of remaining values with NA for height/floors : ', building[['hauteur', 'nombre_d_etages']].isna().all(axis=1).sum())

Number of remaining values with NA for height/floors :  1340


In [13]:
# 2  Basic new indicators : Surface, Compacity, Elongation, Product [1-E].C.S, Product [1-E].S

# Surface
building['area'] = building.geometry.area

# Calculating perimeter
building['perimeter'] = building.geometry.length

# Calculating elongation
building['elongation'] = momepy.Elongation(building).series

# Convexity
building['convexity'] = momepy.Convexity(building).series

# Floor area
building['FA'] = building['nombre_d_etages'] * building['area']

# Product [1-E].C.S
building['ECS'] = (1 - building['elongation']) * building['area'] * building['convexity']

# [1-E].S
building['ES'] = (1 - building['elongation']) * building['area']

# Shared walls
building["SW"] = momepy.SharedWallsRatio(building).series

In [14]:
building.to_file(gpkg, layer="building_morpho", driver="GPKG")